In [1]:
import numpy as np
import matplotlib.pyplot as plt

import time
from copy import deepcopy

import cv2

In [20]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Useful resources
[OpenCV tutorial](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file)

# Capture and handle video

In [3]:
FILE = '../test_data/simple_green.mp4'
cv2.namedWindow('raw_frame')

In [6]:
cap = cv2.VideoCapture(FILE)

In [12]:
def handleFrame(frame, features):
    image = Image(frame)

    for f in features:
        f.refine(image)


def main():
    features = deepcopy([greenCircle])
    while cap.isOpened():
        startTime = time.time()
        ret, frame = cap.read()
        readTime = time.time()

        if frame is None or len(frame) <= 0:
            print "End of video file reached"
            break

        # Do work here
        handleFrame(frame, features)
        
        finishTime = time.time()

        for f in features:
            f.draw(frame)
        scaled = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5)
        cv2.imshow('raw_frame', scaled)

        cv2.waitKey(30)

        print "{}ms reading image, {}ms processing, {}px accessed".format(
            (readTime-startTime)*1000, 
            (finishTime-readTime)*1000, 
            image.pixelsAccessed)
        
main()

hi


NameError: global name 'image' is not defined

# Profile things

In [64]:
ret, frame = cap.read()

def bench():
    features = deepcopy([greenCircle])
    handleFrame(frame, features)
    
%lprun -r -f Circle2D.refine bench()

hi


# Define types for features

In [8]:
class Image(object):
    def __init__(self, img):
        self.img = img
        self.pixelsAccessed = 0
    
    def getHSV(self, x, y):
        self.pixelsAccessed += 1
        rgb = self.img[y:y+1, x:x+1]
        hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)
        return hsv[0, 0]

In [9]:
class Color(object):
    def __init__(self, minimum, maximum, colorspace='HSV'):
        if colorspace != 'HSV':
            raise Exception("Invalid colorspace, only HSV supported")
            
        self.min = minimum
        self.max = maximum
        
    def matches(self, color):
        # Check hue condition
        if self.min[0] > self.max[0]:
            # The hue range wraps past the edge
            if color[0] > self.max[0] and color[0] < self.min[0]:
                return False
        else:
            if color[0] > self.max[0] or color[0] < self.min[0]:
                return False
            
        # Check saturation and value conditions
        for i in (1, 2):
            if color[i] > self.max[i] or color[i] < self.min[i]:
                return False
        
        return True

In [10]:
class Circle2D(object):
    def __init__(self, center, radius, colorRange):
        self.center = center
        self.radius = radius
        self.color  = colorRange
        
        # TODO: this should be a scalar with more reasonable behavior
        self.confidence = True
    
    def draw(self, img):
        """
        Taking in a numpy array representing an OpenCV image,
        draws a visual debugging indication of this feature
        """
        drawColor = (128, 255, 0) if self.confidence else (0, 0, 255)
        cv2.circle(img, self.center, self.radius, drawColor, thickness=5)
        cv2.circle(img, self.center, 5, drawColor, thickness=-1)
        
    def refine(self, image, verbose=False, searchRange = 500):
        """
        Taking in an Image object, updates this feature to better align
        with the observed pixels.
        
        searchRange represents the maximum pixel distance the target
            object could have moved.
        """
        
        def matchesAt(x, y):
            return self.color.matches(image.getHSV(x, y))
        
        if matchesAt(*self.center):
            self.confidence = True
            
        else:
            self.confidence = False
            return

        print 'hi'
        
        ## Find the x-coordinate of the center of the circle
        leftEdge = rightEdge = self.center[0]
        
        maxDist = searchRange + self.radius
        
        missCount = 0
        for i in range(maxDist):
            if not matchesAt(self.center[0]-i, self.center[1]):
                missCount += 1
            else:
                missCount = 0
            
            if missCount > 0:
                leftEdge = self.center[0]-i
                if verbose:
                    print "Left edge found after {} pixels".format(i)
                break
        
        missCount = 0
        for i in range(maxDist):
            if not matchesAt(self.center[0]+i, self.center[1]):
                missCount += 1
            else:
                missCount = 0
            
            if missCount > 0:
                rightEdge = self.center[0]+i
                if verbose:
                    print "Right edge found after {} pixels".format(i)
                break
                
        self.center = (int((leftEdge + rightEdge) / 2), self.center[1])
        
        if verbose:
            print self.center

            print "Center point color is {}: {}".format(
                centerColor,
                'match' if self.color.matches(centerColor) else 'no match'
                )
        
        
        
    
green = Color((40,60,100), (60, 255, 255))
greenCircle = Circle2D((1920/2, 1080/2), 400, green)